In [1]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#time
from datetime import datetime
from datetime import timedelta

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns


# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

In [2]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個 #只留日期
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)
#df_article2 = df_article[['post_time2','title','content']]

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [4]:
# 選那家股票
#company_name = '國巨'
company_name = '奇力新'


# 文章包含那家字
#company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|凱美|同欣電|大毅|君耀|普斯|國巨'
company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|飛磁|旺詮|美磊|美桀|向華科技|奇力新'

# 漲跌幾%
PA = 0.05

# even幾%
PAE = 0.003

In [5]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46


In [6]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,0.0
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,0.0


In [7]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [8]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,80.2


In [9]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,--,80.2


In [10]:
print('fluctuation std = ',df_trend['fluctuation'].std(axis=0))
print('fluctuation mean = ',df_trend['fluctuation'].mean(axis=0))

fluctuation std =  0.030403952190342375
fluctuation mean =  0.0004319276970365839


In [11]:
df_trend.loc[df_trend['fluctuation'] >=  PA, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -PA, 'tag'] = 'down'
df_trend.loc[(df_trend['fluctuation'] >= -PAE) & (df_trend['fluctuation'] <= PAE), 'tag'] = 'even'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,up,80.2


In [12]:
len(df_trend[df_trend['tag']=='up'])

37

In [13]:
len(df_trend[df_trend['tag']=='down'])

39

In [14]:
len(df_trend[df_trend['tag']=='even'])

90

In [15]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
del df_article
df_company.head(2)

10553


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02


In [16]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [17]:
# 'content'全部切詞
corpus = []  # array
for index, row in df_company.iterrows():  
    not_cut = df_company.loc[index,'content']
#     not_cut = row['description']                    # 跟上一行一樣意思
    seg_generator = jieba.cut(not_cut, cut_all=False)  # genarator
    seglist = list(seg_generator)                     # 整篇文章string切出來的list
#    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟
    corpus.append(' '.join(seglist))          # ' '.join(seg_generator)也可

df_company["content2"]=corpus  

Building prefix dict from /media/1tb/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.uf2a0578bcd12a258b328002a38330ff0.cache
Loading model cost 0.450 seconds.
Prefix dict has been built succesfully.


In [18]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01,台股 2015 年 下半年 跌勢 洶洶 ， 全年 跌幅 達 10.14 ％ ， 但 12 檔...
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02,鉅亨網 記者 王莞 甯 台北 台股 2015 年 封關日 最後 一檔 有 神秘 買盤 進...


In [19]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [20]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [21]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [22]:
d.days #只取天數

1

In [23]:
int(d.days) #幾天 轉整數

1

In [24]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [25]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
999,Moneydj理財網,0,1.週三台股在美股重挫以及陸股再度啟動融斷機制之下，指數重挫點，指數跌破十年線，所幸尾盤大型...,1452156767683_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 16:29:00,重大要聞,yahoo股市,日盛投顧：台股短線有反彈機會,2016-01-07,1 . 週三 台股 在 美股 重挫 以及 陸股 再度 啟動 融斷 機制 之下 ， 指數 重挫...


In [26]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [27]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [28]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
df_company['label5566']='--'
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label5566'] = df_trend.loc[index,'tag']
    except:
        continue

In [29]:
print(len(df_company[df_company['label5566']=='down']))
df_company[df_company['label5566']=='down'].head(2)

1284


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
1522,財訊快報,0,◆全台勞檢啟動，11類行業小心，其中284家電子零組件企業，實施全面普查。<BR> ◆海外分...,1452472319234_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:05:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-11,◆ 全台 勞檢 啟動 ， 11 類 行業 小心 ， 其中 284 家 電子 零組件 企業 ，...,down
1530,時報資訊,0,陸股暫停熔斷機制，減緩市場對於系統性風險的疑慮，法人指出，台股短線壓力仍大，若結構出現惡化現...,1452472329341_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:15:00,重大要聞,yahoo股市,《基金》全球經濟趨緩，慎選台股標的,2016-01-11,陸股 暫停 熔斷 機制 ， 減緩 市場 對於 系統性 風險 的 疑慮 ， 法人 指出 ， 台...,down


In [30]:
print(len(df_company[df_company['label5566']=='up']))
df_company[df_company['label5566']=='up'].head(2)

891


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-04,◆ 千億 科技 預算 ， 張善政 喊 加碼 ， 主計 總處 初步 匡列 1 061 億 ...,up
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,2016-01-04,2016 年開 紅盤 日 ， 台股 狂洩 逾 200 點 ！ 儘管 2015 年 最後 一個...,up


In [31]:
print(len(df_company[df_company['label5566']=='even']))
df_company[df_company['label5566']=='even'].head(2)

911


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
3121,鉅亨網,0,受到月線反壓、外資籌碼面持續偏保守的影響下，台股期現貨今(20)日同步下壓，加權指數開低走低...,1453258680867_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 10:42:00,重大要聞,yahoo股市,台股盤中─個股漲勢稀落 指數一度大跌百點 台指期難拉高結算,2016-01-20,受到 月線 反壓 、 外資 籌碼面 持續 偏 保守 的 影響 下 ， 台股 期 現貨 今 (...,even
3192,Moneydj理財網,0,1.歐美股市走穩，不過受期指結算影響，台股週三開低，加上港股重挫，指數震盪走低，盤面上部份生...,1453281180692_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 16:16:00,重大要聞,yahoo股市,日盛投顧：台股短線參考區間7600-7900,2016-01-20,1 . 歐美 股市 走穩 ， 不過 受 期指 結算 影響 ， 台股 週三 開低 ， 加上 港...,even


In [32]:
df_company = df_company[df_company['label5566'].str.contains('up|down|even')]

In [33]:
import re
features = []   # features=list()
with open('finance.words.txt', 'r',encoding='utf-8') as data:
    for line in data:
#        line = re.sub('[a-zA-Z0-9\W]', '', line)                     # 把數字英文去掉
        line = re.sub('[0-9]', '', line)                              # 把數字去掉
        features.append(line.replace('\n', '').replace(' ', ''))      # 空格 \n去掉

print(len(features))
print(type(features))
features[:10]

3403
<class 'list'>


['營收', '交易', '成長', '上漲', '外資', '法人', '增加', '下跌', '獲利', '股']

In [34]:
#df_keyword1 = pd.read_csv("final_higher_tf_idf_part.csv",encoding="utf-8") #上漲形容詞
#df_keyword2 = pd.read_csv("final_lower_tf_idf_part.csv",encoding="utf-8")  #下跌形容詞
#df_keyword = pd.concat([df_keyword1,df_keyword2])
#del df_keyword1,df_keyword2
#df_keyword.head()

In [35]:
#features = df_keyword['key'].to_numpy()
#features

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

#features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
#features = features[:1000]
cv = TfidfVectorizer()   #預設有空格就一個feature
#cv = TfidfVectorizer(features) # 設定自己要的詞 

r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
                       df_company.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

r.fillna(value=0.0, inplace=True)
r.head(2)

,00,000,000000001,000000002,000000003,000000004,000000005,000000006,000000007,000000008,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [37]:
# from sklearn.feature_extraction.text import CountVectorizer

# #features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
# #features = features[:1000]
# #cv = CountVectorizer()   #預設有空格就一個feature
# cv = CountVectorizer(vocabulary = features) # 設定自己要的詞 

# r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
#                        df_company.index,
#                        cv.get_feature_names(), 
#                        default_fill_value=0)

# r.head(2)

In [38]:
df_company2 = pd.concat([df_company,r], axis=1)
df_company2.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [39]:
#df_company2 = df_company2[df_company2['label5566'].str.contains('up|down|even')]   #只取漲跌
Y = df_company2['label5566']
X = df_company2.drop(columns=['author','comment_count','content','id','p_type','page_url','post_time','s_area_name','s_name','title','post_time2','content2','label5566'])
X.fillna(value=0.0, inplace=True)
X.head(2)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,00,000,000000001,000000002,000000003,000000004,000000005,000000006,000000007,000000008,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [40]:
Y.head(2)

222    up
289    up
Name: label5566, dtype: object

In [41]:
X = X.to_numpy()
Y = Y.to_numpy()

In [42]:
#將X:features array, Y:lable array 都切成 1:4

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed, stratify=Y ) 
# 設定 stratify = Y 把每個類別平均
print(len(X_train),len(X_validation))

2468 618


In [43]:
model_RandomForest = RandomForestClassifier()
model_RandomForest.fit(X_train, Y_train)
print(model_RandomForest.score(X_train, Y_train))

predictions = model_RandomForest.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.986223662884927
0.5161812297734628
[[188  29  40]
 [ 88  76  19]
 [ 85  38  55]]
              precision    recall  f1-score   support

        down       0.52      0.73      0.61       257
        even       0.53      0.42      0.47       183
          up       0.48      0.31      0.38       178

   micro avg       0.52      0.52      0.52       618
   macro avg       0.51      0.49      0.48       618
weighted avg       0.51      0.52      0.50       618



In [44]:
model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X_train, Y_train)
print(model_DecisionTree.score(X_train, Y_train))

predictions = model_DecisionTree.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9991896272285251
0.49838187702265374
[[162  42  53]
 [ 66  79  38]
 [ 58  53  67]]
              precision    recall  f1-score   support

        down       0.57      0.63      0.60       257
        even       0.45      0.43      0.44       183
          up       0.42      0.38      0.40       178

   micro avg       0.50      0.50      0.50       618
   macro avg       0.48      0.48      0.48       618
weighted avg       0.49      0.50      0.49       618



In [45]:
model_GaussianNB = GaussianNB()
model_GaussianNB.fit(X_train, Y_train)
print(model_GaussianNB.score(X_train, Y_train))

predictions = model_GaussianNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9817666126418152
0.6035598705501618
[[178  37  42]
 [ 49 101  33]
 [ 57  27  94]]
              precision    recall  f1-score   support

        down       0.63      0.69      0.66       257
        even       0.61      0.55      0.58       183
          up       0.56      0.53      0.54       178

   micro avg       0.60      0.60      0.60       618
   macro avg       0.60      0.59      0.59       618
weighted avg       0.60      0.60      0.60       618



In [46]:
model_MultinomialNB = MultinomialNB()
model_MultinomialNB.fit(X_train, Y_train)
print(model_MultinomialNB.score(X_train, Y_train) )

predictions = model_MultinomialNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_MultinomialNB.coef_)

0.49189627228525123
0.4320388349514563
[[257   0   0]
 [173  10   0]
 [175   3   0]]
              precision    recall  f1-score   support

        down       0.42      1.00      0.60       257
        even       0.77      0.05      0.10       183
          up       0.00      0.00      0.00       178

   micro avg       0.43      0.43      0.43       618
   macro avg       0.40      0.35      0.23       618
weighted avg       0.40      0.43      0.28       618

[[ -8.47359068  -8.63953867 -10.84762262 ... -10.78610467 -10.3756694
  -10.84762262]
 [ -8.39788254  -9.03565822 -10.79244699 ... -10.79244699 -10.38647168
  -10.79244699]
 [ -9.04710786  -8.91940686 -10.7859615  ... -10.7859615  -10.66900348
  -10.7859615 ]]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
# 用10-Fold CV並且列出平均的效率
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
# 呼叫單個model MLP
model_LogisticRegression = LogisticRegression()
name = 'LogisticRegression'
kfold = model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_validate(model_LogisticRegression, X, Y, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results['test_score'])
print("%s: %f (%f)" % (name, cv_results['test_score'].mean(), cv_results['test_score'].std()))

print(cv_results['train_score'])
print(cv_results['fit_time'])
print(cv_results['score_time'])

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/barry/minico

[0.58319871 0.61102107 0.60615883 0.6012966  0.60064935]
LogisticRegression: 0.600465 (0.009410)
[0.8686664  0.86391252 0.86269745 0.86674767 0.86356275]
[4.93448734 3.87944627 3.91621566 4.058465   3.96334434]
[0.0460453  0.0308888  0.04823804 0.04085231 0.03393149]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [48]:
model_LogisticRegression = LogisticRegression()
model_LogisticRegression.fit(X_train, Y_train)
print(model_LogisticRegression.score(X_train, Y_train))

predictions = model_LogisticRegression.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LogisticRegression.coef_)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8638573743922204
0.6084142394822006
[[223  11  23]
 [ 78  94  11]
 [ 84  35  59]]
              precision    recall  f1-score   support

        down       0.58      0.87      0.69       257
        even       0.67      0.51      0.58       183
          up       0.63      0.33      0.44       178

   micro avg       0.61      0.61      0.61       618
   macro avg       0.63      0.57      0.57       618
weighted avg       0.62      0.61      0.59       618

[[-0.18924348 -0.35295836  0.         ...  0.0365238   0.14378688
   0.        ]
 [ 0.69652592  0.18673648  0.         ... -0.02953141  0.04521966
   0.        ]
 [-0.54909781  0.18528106  0.         ... -0.00848663 -0.18551765
   0.        ]]


In [49]:
# # 用10-Fold CV並且列出平均的效率


# # 呼叫單個model MLP
# model_svclinear = SVC(kernel='linear')
# name = 'SVC'
# kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

# cv_results = model_selection.cross_val_score(model_svclinear, X_train, Y_train, cv=kfold, scoring='accuracy')
# #model用MLP()                cross valitation 

# print(cv_results)
# print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

In [50]:
# model_svclinear = SVC(kernel='linear')
# model_svclinear.fit(X_train, Y_train)
# print(model_svclinear.score(X_train, Y_train))
# predictions = model_svclinear.predict(X_validation)
# print(accuracy_score(Y_validation, predictions))
# print(confusion_matrix(Y_validation, predictions))
# print(classification_report(Y_validation, predictions))
# print(model_svclinear.coef_)

In [51]:
# 用10-Fold CV並且列出平均的效率
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
# 呼叫單個model MLP
from sklearn.svm import LinearSVC
model_LinearSVC = LinearSVC()
name = 'LinearSVC'
kfold = model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_validate(model_LinearSVC, X, Y, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results['test_score'])
print("%s: %f (%f)" % (name, cv_results['test_score'].mean(), cv_results['test_score'].std()))

print(cv_results['train_score'])
print(cv_results['fit_time'])
print(cv_results['score_time'])

[0.64458805 0.64829822 0.67585089 0.64505673 0.63798701]
LinearSVC: 0.650356 (0.013180)
[0.98702878 0.9890644  0.98784933 0.98744431 0.98623482]
[3.81608677 3.79883051 3.83606958 3.7911191  3.81960678]
[0.02869987 0.02805901 0.04066586 0.03562808 0.03275776]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [52]:
from sklearn.svm import LinearSVC
model_LinearSVC = LinearSVC()
model_LinearSVC.fit(X_train, Y_train)
print(model_LinearSVC.score(X_train, Y_train))
predictions = model_LinearSVC.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LinearSVC.coef_)

0.986223662884927
0.63915857605178
[[197  21  39]
 [ 48 114  21]
 [ 56  38  84]]
              precision    recall  f1-score   support

        down       0.65      0.77      0.71       257
        even       0.66      0.62      0.64       183
          up       0.58      0.47      0.52       178

   micro avg       0.64      0.64      0.64       618
   macro avg       0.63      0.62      0.62       618
weighted avg       0.64      0.64      0.63       618

[[ 0.02097546 -0.45265782  0.         ...  0.10039975  0.28100748
   0.        ]
 [ 0.14465809  0.46268549  0.         ... -0.10973311 -0.01648837
   0.        ]
 [-0.12778659 -0.00425823  0.         ...  0.         -0.2842639
   0.        ]]


In [53]:
from sklearn.linear_model import SGDClassifier

# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_SGDClassifier = SGDClassifier(loss='hinge')
#model_SGDClassifier = SGDClassifier(loss='log')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_SGDClassifier, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDCla

[0.60323887 0.58299595 0.60121457 0.5841785  0.65314402]
SVC: 0.604954 (0.025507)


In [54]:
from sklearn.linear_model import SGDClassifier
model_SGDClassifier = SGDClassifier(loss='hinge')
model_SGDClassifier.fit(X_train, Y_train)
print(model_SGDClassifier.score(X_train, Y_train))
predictions = model_SGDClassifier.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_SGDClassifier.coef_)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.973257698541329
0.63915857605178
[[186  43  28]
 [ 34 137  12]
 [ 39  67  72]]
              precision    recall  f1-score   support

        down       0.72      0.72      0.72       257
        even       0.55      0.75      0.64       183
          up       0.64      0.40      0.50       178

   micro avg       0.64      0.64      0.64       618
   macro avg       0.64      0.63      0.62       618
weighted avg       0.65      0.64      0.63       618

[[-0.28395778 -1.73170296  0.         ...  0.19026788  0.78594076
   0.        ]
 [ 0.29897121  1.21410303  0.         ... -0.23783485  0.07164148
   0.        ]
 [-0.87936115 -0.19905737  0.         ...  0.         -0.66780611
   0.        ]]


In [55]:
model_MLP = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200)
model_MLP.fit(X_train, Y_train)
print(model_MLP.score(X_train, Y_train))

predictions = model_MLP.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
#print(model_MLP.coefs_)

0.9987844408427877
0.6634304207119741
[[194  27  36]
 [ 40 117  26]
 [ 39  40  99]]
              precision    recall  f1-score   support

        down       0.71      0.75      0.73       257
        even       0.64      0.64      0.64       183
          up       0.61      0.56      0.58       178

   micro avg       0.66      0.66      0.66       618
   macro avg       0.65      0.65      0.65       618
weighted avg       0.66      0.66      0.66       618

[array([[-3.12894328e-02,  2.74705129e-02, -2.87448752e-02, ...,
         7.95641873e-03, -3.81332536e-02,  2.26636841e-02],
       [ 5.32778843e-03,  4.85740264e-02, -1.03079087e-02, ...,
         2.77132826e-02, -4.21465948e-03, -4.40034749e-03],
       [ 3.49316281e-29, -3.88484131e-31,  4.63057943e-29, ...,
         7.00457546e-30, -3.43159204e-30, -4.39369712e-29],
       ...,
       [ 7.62908840e-02, -7.82956863e-02,  7.08711157e-02, ...,
        -5.92281767e-02,  7.05957266e-02,  3.59873726e-02],
       [-5.90652781e-02,  

In [56]:
# 用10-Fold CV並且列出平均的效率
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
# 呼叫單個model MLP
model_MLP = MLPClassifier(hidden_layer_sizes=(100, ),max_iter=200)
name = 'MLP'
kfold = model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_validate(model_MLP, X, Y, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results['test_score'])
print("%s: %f (%f)" % (name, cv_results['test_score'].mean(), cv_results['test_score'].std()))

print(cv_results['train_score'])
print(cv_results['fit_time'])
print(cv_results['score_time'])

[0.64620355 0.65153971 0.6726094  0.68881686 0.65746753]
MLP: 0.663327 (0.015507)
[0.99878395 0.99918996 0.99837991 0.99878493 0.99878543]
[288.60743809 266.57465577 163.71603084 169.15406442 193.94162226]
[0.06605864 0.0471735  0.0339725  0.0347867  0.03366089]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [ ]:
cv.get_feature_names()[-100:] # 尾巴100個